# IFC Export using Autodesk Automation API

This notebook demonstrates how to export IFC files from Revit models using the Autodesk Platform Services (APS) Design Automation API.

**Important:** This example embeds JSON configuration data directly in the work item payload using `ActivityJsonParameter`. This approach works well for smaller JSON files (under 60KB). For larger JSON files, consider uploading them to storage instead (see example 03_structural_elements).

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/#additional-notes

This example follows the official tutorial: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/

**Note:** The input Revit file, JSON export settings, and compiled app bundle can be found in `examples\02_ifc_export\files`

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [1]:
import os
import json
import logging
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    ActivityJsonParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [2]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

Authentication successful. Nickname: viktortest


## Step 3: Define Project Constants

Set up all the constants needed for the IFC export workflow.

In [3]:
# Define constants
app_bundle_name = "IfcExportBundle"
activity_name = "RevitIfcExportAppActivity"
alias = "prod"
bucket_key = "vkt001_bucket_ifc_export"
zip_path = Path.cwd() / "files" / "IfcExportDA.bundle.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")

App Bundle: viktortest.IfcExportBundle+prod
Activity: viktortest.RevitIfcExportAppActivity+prod


## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the IFC export logic.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step4-publish-appbundle/

In [4]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.Revit+2024",
    alias=alias,
    zip_path=str(zip_path),
    description="Exports IFC from selected views",
)

bundle.deploy(token)
print("App bundle deployed successfully!")

App bundle deployed successfully!


## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Note:** This example uses `ActivityJsonParameter` to embed JSON settings directly in the work item payload. This is suitable for configuration data under 60KB.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/#step-1-create-a-new-activity

In [5]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Zipped IFC files
output_zip = ActivityOutputParameter(
    name="result",
    localName="result",
    verb="put",
    description="Zipped IFCs",
    zip=True,
    bucketKey=bucket_key,
    objectKey="IFCExport.zip",
)

# Input: JSON settings
input_json = ActivityJsonParameter(
    name="ifcSettings",
    localName="ifc_settings.json",
    verb="get",
    description="Export Settings Parameter JSON",
)

print("Activity parameters defined successfully!")

Activity parameters defined successfully!


## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/

In [6]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_zip, input_json],
    engine="Autodesk.Revit+2024",
    appbundle_full_name=appbundle_full_alias,
    description="Export IFC from views using JSON settings",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print("Activity deployed successfully!")

Activity deployed successfully!


## Step 7: Upload Input Revit File

Upload the Revit file to Object Storage Service (OSS).

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step6-prepare-cloud-storage/

In [7]:
input_rvt_path = Path.cwd() / "files" / "SteelStructureFrame.rvt"
input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"Input Revit file uploaded: {input_rvt_path.name}")

Input Revit file uploaded: SteelStructureFrame.rvt


## Step 8: Load and Set IFC Export Settings

Read the IFC export settings from JSON and set them for the work item.

**Note:** The JSON content will be embedded directly in the work item payload using `ActivityJsonParameter.set_content()`. This is appropriate for smaller configuration files.

In [8]:
settings_path = Path.cwd() / "files" / "export_settings.json"
with settings_path.open("r", encoding="utf-8") as f:
    settings = json.load(f)

input_json.set_content(settings)
print(f"IFC settings loaded from: {settings_path.name}")
print(f"Settings: {json.dumps(settings, indent=2)}")

IFC settings loaded from: export_settings.json
Settings: {
  "view_names": [
    "Foundations"
  ],
  "FileVersion": "IFC4",
  "IFCFileType": "IFC",
  "ExportBaseQuantities": true,
  "SpaceBoundaryLevel": 2,
  "FamilyMappingFile": "C:\\Data\\ifc_mapping.txt",
  "ExportInternalRevitPropertySets": false,
  "ExportIFCCommonPropertySets": true,
  "ExportAnnotations": false,
  "Export2DElements": false,
  "ExportRoomsInView": false,
  "VisibleElementsOfCurrentView": false,
  "ExportLinkedFiles": false,
  "IncludeSteelElements": false,
  "ExportPartsAsBuildingElements": true,
  "UseActiveViewGeometry": false,
  "UseFamilyAndTypeNameForReference": false,
  "Use2DRoomBoundaryForVolume": false,
  "IncludeSiteElevation": false,
  "ExportBoundingBox": false,
  "ExportSolidModelRep": false,
  "StoreIFCGUID": false,
  "ExportSchedulesAsPsets": false,
  "ExportSpecificSchedules": false,
  "ExportUserDefinedPsets": false,
  "ExportUserDefinedPsetsFileName": "",
  "ExportUserDefinedParameterMapping": 

## Step 9: Create and Execute Work Item

Create a work item with all parameters and execute it.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [9]:
work_item = WorkItem(
    parameters=[input_revit, output_zip, input_json],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")

Work item created. Starting execution...


## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [10]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print(f"Work item completed with status: {last_status}")
print(f"Full response: {json.dumps(status_resp, indent=2)}")

2025-11-02 23:57:16,010 INFO Polling work item status, id=2b13f73ecdda4947a8b6255c7c58abe4
2025-11-02 23:57:16,868 INFO [  0s] status=inprogress report_url=None
2025-11-02 23:57:27,718 INFO [ 10s] status=inprogress report_url=None
2025-11-02 23:57:38,579 INFO [ 20s] status=inprogress report_url=None
2025-11-02 23:57:49,397 INFO [ 30s] status=inprogress report_url=None
2025-11-02 23:58:00,204 INFO [ 40s] status=inprogress report_url=None
2025-11-02 23:58:11,020 INFO [ 50s] status=inprogress report_url=None
2025-11-02 23:58:21,792 INFO [ 60s] status=inprogress report_url=None
2025-11-02 23:58:32,676 INFO [ 70s] status=success report_url=https://dasprod-store.s3.amazonaws.com/workItem/viktortest/2b13f73ecdda4947a8b6255c7c58abe4/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEI3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCID96Qv4cU1Y4wcqZsr7Fl9hoalACsyR7OwEHNi5z%2FlZbAiA%2Bd6bttaC0BRCF9t0Xe01XEQQ1n%2BWxQnUUY1tqng5zbSqAAwhVEAQaDDIyMDQ3MzE1MjMxMCIMUAzpD1szrWWxotCtKt0C

Work item completed with status: success
Full response: {
  "status": "success",
  "reportUrl": "https://dasprod-store.s3.amazonaws.com/workItem/viktortest/2b13f73ecdda4947a8b6255c7c58abe4/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEI3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCID96Qv4cU1Y4wcqZsr7Fl9hoalACsyR7OwEHNi5z%2FlZbAiA%2Bd6bttaC0BRCF9t0Xe01XEQQ1n%2BWxQnUUY1tqng5zbSqAAwhVEAQaDDIyMDQ3MzE1MjMxMCIMUAzpD1szrWWxotCtKt0C2Y7H5YkMlj7wCW5D2asb9LbRe9wJ%2F8S9fVx8CnAJ6i840i9AGcPr2bzzv%2Bnp5GsXukOOLQ9GMYQ%2BDMTmSj3sZscPS%2FJhvsa%2FuLonNoiwZeoFQRA1vx173uOVj6lkzWSnBsSDVs3G%2FQM%2Bp9d6Q0WxLXDlOrVwdKZ7jOAv5OwD2k%2F%2FfGb4IaY17yaWkWa57ke0r07wK9bKWgSKSD1AM2%2FDwHIQzeZxYxvA3A0HohqV2wvntrHDZbXmeOebiyNO3sbRwXy%2BM9ymKLKlq36h3YaqJdowagZ%2F8xbnDRoEIARw5CAUYwxhrz7GaHfF14gac4xTvbGOjO50rfucrHHV%2FJCR468%2FwhGxqm8dxTi6xJyp%2FDh49blJT8viryrev2RFtP1SObe7Bi8vx8swdiK0GAav0MbtFmLVgVMG1LAKidxMwkd22RKcKPzMwTg%2BlnhoPA5%2ByvUuRbZbo4qu4gj4PTCx2aDIBjqfAbqx3mQOYvvqBADZ6qZZoXw%2BwBghW7ii27

## Step 11: Download Results

If successful, download the IFC export results.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [11]:
if last_status == "success":
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_zip = out_dir / "IFCExport.zip"
    output_zip.download_to(output_path=str(out_zip), token=token)
    print(f"✅ Download successful: {out_zip}")
else:
    print(f"❌ Work item failed or did not complete successfully. Status: {last_status}")

✅ Download successful: c:\Users\aleja\Documents\aps_automation_sdk\examples\02_ifc_export\files\output\IFCExport.zip


## Step 12: Cleanup Resources

Delete the activity and app bundle to clean up resources.

In [12]:
print("Cleaning up: deleting activity and app bundle...")

try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✅ Activity deleted: {activity_full_alias}")
except Exception as e:
    print(f"⚠️ Failed to delete activity: {e}")

try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✅ App bundle deleted: {app_bundle_name}")
except Exception as e:
    print(f"⚠️ Failed to delete app bundle: {e}")

print("\n🎉 IFC Export workflow completed!")

Cleaning up: deleting activity and app bundle...
✅ Activity deleted: viktortest.RevitIfcExportAppActivity+prod
✅ App bundle deleted: IfcExportBundle

🎉 IFC Export workflow completed!
